In [1]:
library(variableStars)
library(fields)
library(keras)

Loading required package: spam
Loading required package: dotCall64
Loading required package: grid
Spam version 2.2-0 (2018-06-19) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: 'spam'

The following objects are masked from 'package:base':

    backsolve, forwardsolve

Loading required package: maps
See www.image.ucar.edu/~nychka/Fields for
 a vignette and other supplements. 


In [2]:
compute_star <- function(data, real_dnu, numFrequencies=30) {


  # Execute experiment
  result <- process(
    frequency = data$V1,
    amplitude = data$V2,
    filter = "uniform",
    gRegimen = 0,
    maxDnu = 1,
    minDnu = 15,
    numFrequencies = numFrequencies,
    dnuGuessError = -1,
    debug = F,
    processFirstRangeOnly = numFrequencies
  )
  
  
    # Save ft of diffs
  ftS <-
    stats.bin(as.numeric(result$fresAmps[[names(result$fresAmps)[1]]]$fInv),
              as.numeric(result$fresAmps[[names(result$fresAmps)[1]]]$b),
              breaks = cuts_breaks)$stats
  ft_1D <- ftS[8, 1:(length(cuts_breaks) - 1)]
  ft_1D[is.na(ft_1D)] <- 0

    diffS <-
      stats.bin(
        as.numeric(result$diffHistogram$histogram$bins),
        as.numeric(result$diffHistogram$histogram$values),
        breaks = cuts_breaks
      )$stats
    diff_2D <- diffS[8, 1:(length(cuts_breaks) - 1)]
    diff_2D[is.na(diff_2D)] <- 0
    
    # Save crosscorrelation
    cross <- stats.bin(
      as.numeric(result$crossCorrelation$index),
      as.numeric(result$crossCorrelation$autocorre),
      breaks = cuts_breaks
    )$stats
    cross_3D <- cross[8, 1:(length(cuts_breaks) - 1)]
    cross_3D[is.na(cross_3D)] <- 0
          
    # Raw information
    rawS <-
      stats.bin(as.numeric(data$V1),
                as.numeric(data$V2),
                breaks = cuts_breaks)$stats
    raw_1D <- rawS[8, 1:(length(cuts_breaks) - 1)]
    raw_1D[is.na(raw_1D)] <- 0
    
    # Assert all dimensions are equal
    stopifnot((length(ft_1D) == length(diff_2D)) ==
                ((length(diff_2D) == length(cross_3D)) ==
                   (
                     length(cross_3D) == length(cuts_breaks) - 1
                   )))
    

  rows <- 506823
  cols <- (1626 - 2) / 4
  dimensions <- 4 # Number of channels
  X_scuti <- array(0, c(1, cols, 4))
  # Y train is a 1D matrix with rows and targets
  Y <- matrix(0, nrow = 1, ncol = num_classes)
  ind_data <- seq(from = 1, to = rows)
  
  X_scuti[1, , 1] <- ft_1D
  X_scuti[1, , 2] <- diff_2D
  X_scuti[1, , 3] <- cross_3D
  X_scuti[1, , 4] <- raw_1D
    
  #Y_scuti <- to_categorical(real_dnu, num_classes)
  Y_scuti <- real_dnu
    
  return(list("X_scuti"=X_scuti,"Y_scuti"=Y_scuti))
}

In [3]:
# Resolution for target frequencY_scuti [0-100]
input_resolution <- 0.25
output_resolution <- 1.0
# Input dimension
cuts_breaks <- c(-Inf, seq(0, 101, input_resolution), Inf)
input_dim <- length(cuts_breaks) - 1
# Output dimension
num_classes <-
  length(seq(
    from = 0.1,
    to = 14 / 0.0864,
    by = output_resolution
  )) # Buckets of possible classes



# MatriX_scuti to save stars

cols <- (1626 - 2) / 4
dimensions <- 4 # Number of channels
X_scuti <- array(0, c(11, cols, 4))
Y_scuti <- matrix(0, nrow = 11, ncol = num_classes)

stars_base_dir <- "~/Projects/variableStars/data/deltaScuti/"


res <- compute_star(read.csv(paste0(stars_base_dir,"CID100866999.lis"), sep="", header=F), 30, 56)
X_scuti[1,,1] <- res$X_scuti[1,,1];
Y_scuti[1] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"CID105906206.lis"), sep="", header=F), 30, 20)
X_scuti[2,,] <- res$X_scuti[1,,]
Y_scuti[2] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"HD15082.lis"), sep="", header=F), 30, 80)
X_scuti[3,,] <- res$X_scuti[1,,]
Y_scuti[3] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"HD159561.lis"), sep="", header=F), 30, 38)
X_scuti[4,,] <- res$X_scuti[1,,]
Y_scuti[4] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"HD172189.lis"), sep="", header=F), 30, 19)
X_scuti[5,,] <- res$X_scuti[1,,]
Y_scuti[5] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"KIC10080943.lis"), sep="", header=F), 30, 52)
X_scuti[6,,] <- res$X_scuti[1,,]
Y_scuti[6] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"kic10661783.lis"), sep="", header=F), 30, 39)
X_scuti[7,,] <- res$X_scuti[1,,]
Y_scuti[7] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"KIC3858884.lis"), sep="", header=F), 30, 29)
X_scuti[8,,] <- res$X_scuti[1,,]
Y_scuti[8] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"kic4544587.lis"), sep="", header=F), 30, 74)
X_scuti[9,,] <- res$X_scuti[1,,]
Y_scuti[9] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"KIC8262223.lis"), sep="", header=F), 30, 77)
X_scuti[10,,] <- res$X_scuti[1,,]
Y_scuti[10] <- res$Y_scuti[1]

res <- compute_star(read.csv(paste0(stars_base_dir,"KIC9851944.lis"), sep="", header=F), 30, 26)
X_scuti[11,,] <- res$X_scuti[1,,]
Y_scuti[11] <- res$Y_scuti[1]

In [5]:
save(X_scuti, file = "X_scuti.rda")
save(Y_scuti, file = "Y_scuti.rda")